In [163]:
import PyPDF2
import re
import os
from chemdataextractor import Document

In [20]:
list_of_files = os.listdir('./sample_methods')

In [21]:
list_of_files

['100011.pdf',
 '101212-E17D.pdf',
 '100024.pdf',
 '102452-E10B.pdf',
 '101931-E07B.pdf',
 '100077.pdf',
 '102557-E11A.pdf',
 '102668-E18B.pdf',
 '102819-E17A.pdf']

In [22]:
def pdf_parser(filepath):
    filename = filepath
    pdfFileObj = open(filename,'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
        
    if text != "":
        text = text
    
#     else:
#         text = textract.process(fileurl, method='tesseract', language='eng')
    
    return text


In [23]:
text_list = [pdf_parser('sample_methods/' + i) for i in list_of_files]

* __Deleted file no. 5__

In [24]:
for idx, i in enumerate(text_list):
    if i == '':
        del text_list[idx]

## Document ID

In [25]:
doc_ids = [re.search(r'\d{6,7}\s{0,5}-\s{0,5}\w{0,1}\s{0,5}\d{0,2}\s{0,5}\w{0,2}\s{0,5}B{0,1}', i, flags=0).group() 
           if re.search(r'\d{6,7}\s{0,5}-\s{0,5}\w{0,1}\s{0,5}\d{0,2}\s{0,5}\w{0,1}\s{0,5}B{0,1}', i, flags=0) != None 
           else re.search(r'\d{6,7}', i, flags=0).group() for i in text_list]

doc_ids = [re.sub(r'[\n\s+]', '', i) for i in doc_ids]

doc_ids

['100011',
 '101212-E17D',
 '100024',
 '102452-E10B',
 '101931-E07B',
 '102557-E11A',
 '102668-E18B',
 '102819-E17A']

## Extract Scope

In [90]:
def scope_extractor(text):
    scope_pattern = re.search(r'this\smethod\sis', text.lower(), flags=0)
    next_sec_pattern = re.search(r'\d.\s+\w+.', text[scope_pattern.start():], flags=0)
    return text[scope_pattern.start():scope_pattern.start() + next_sec_pattern.start()]

In [99]:
extracted_scopes = [scope_extractor(i) for i in text_list]

In [179]:
print(extracted_scopes[5])

This method is applicable to the analysis of 
benzyl alcohol in 
low density 
polyethylene 
pellets 
over 
the range o
f 0.


In [180]:
a = Document(extracted_scopes[5])

a.cems

[Span('benzyl alcohol', 46, 60), Span('low density \npolyethylene', 65, 90)]

## Extract Precision

In [154]:
def precision_extractor(text):
    precision_pattern = re.search(r'\d{1,2}.\s+Pre\s{0,1}cision', text, flags=0)
    next_sec_pattern = re.search(r'\d{1,2}\.\s+\w+.', text[precision_pattern.end():], flags=0)
    return text[precision_pattern.start():precision_pattern.end() + next_sec_pattern.start()]

In [162]:
precision_extractor(text_list[7])

'12.\n \nPre\ncision\n \n12.1\n \nPrecision \nhas been \ndetermined from \nmultiple analyses o\nf\n \ntwo \npowder \nbatches\n \n[n = 2\n0]\n \nand \none \npellet \nbatch\n \n[n = 1\n0]\n.  The \naverage \nvolatiles \ncontent \nranged from \n0.60\n \nto 0.81% (w/w).  \nThe precision data indicate a pooled standard deviation [s\npooled\n] of \n\n \n0.022\n% and pooled \nrelative standard deviation \n(RSD) \n[\nRSDp\nooled\n] \nof \n \n\n \n3.\n1\n%. \n \n12.2\n \nAt the \n95% confidence\n \nlevel\n, individual measurements on similar samples may vary from \nthe \nlong\n-\nterm average\n \nby \n\n \n0.045\n% \n[t × s\npooled\n, where t = t\n-\nvalue \nof 2.05\n \nat \n27\n \ndegrees of \nfreedom\n]\n \nand \n\n \n6.\n4\n% relative [t × \nRSDpooled\n, where t = t\n-\nvalue \nof \n'

In [158]:
extracted_precision = [precision_extractor(i) for i in text_list]

In [160]:
extracted_precision[5]

'12. Precision    Precision has been determined \nfrom multiple analyses [n = \n20] of two \nsynthetic \nprepared samples of \nbenzyl alcohol in water\n.  The synthetic samples were prepared fresh daily (Section 4.1). \nThe analyses \nwere performed on two separate days at an average benzyl alcohol\n concentration\n [x] of \n0.09 ppm \n(g/g).  The precision data indicate a standard deviation [\ns] of \n0.01\n ppm (\ng/g\n), where s = standard \ndeviation of the validation data.\n  \n The estimated prediction interval at the 95% confidence level of a future final result determined on a \nsimilar sample [±\n t(n-1) × s; where t\n(n-1)   = 2.093 =\n t-value at (n\n-1) degrees of freedom] is ± \n0.01 ppm \n(g/g)\n.  This assumes a normal distribution of results and equal variability between locations.\n  Any future final result obtained on a similar sample (with a true \nbenz\nyl alcohol concentration\n [x] of \n0.090 ppm (\ng/g)\n) is expected to range from \n0.08 to 0.10 \nppm (\ng/g) \n